In [1]:
import os, math, subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# some settings for displaying Pandas results
pd.set_option('display.width', 2000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.precision', 4)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_colwidth', -1)

# Load data and exploring statistics

In [2]:
!ls -lah home-credit-default-risk/

total 2.5G
drwxrwsr-x 2 hong ubuntu 4.0K Apr 16 16:38 .
drwxrwsr-x 4 hong ubuntu 4.0K Apr 16 16:39 ..
-rw-rw-r-- 1 hong ubuntu  26M Jun 26  2018 application_test.csv
-rw-rw-r-- 1 hong ubuntu 159M Jun 26  2018 application_train.csv
-rw-rw-r-- 1 hong ubuntu 359M Jun 26  2018 bureau_balance.csv
-rw-rw-r-- 1 hong ubuntu 163M Jun 26  2018 bureau.csv
-rw-rw-r-- 1 hong ubuntu 405M Jun 26  2018 credit_card_balance.csv
-rw-rw-r-- 1 hong ubuntu  37K Jun 26  2018 HomeCredit_columns_description.csv
-rw-rw-r-- 1 hong ubuntu 690M Jun 26  2018 installments_payments.csv
-rw-rw-r-- 1 hong ubuntu 375M Jun 26  2018 POS_CASH_balance.csv
-rw-rw-r-- 1 hong ubuntu 387M Jun 26  2018 previous_application.csv
-rw-rw-r-- 1 hong ubuntu 524K Jun 26  2018 sample_submission.csv


In [3]:
ls_files = subprocess.check_output(["ls", "-lah", "home-credit-default-risk/"]).decode("utf-8").splitlines()[3:]
ls_files = [f.split()[-1] for f in ls_files]
ls_files = sorted([f for f in ls_files if f not in ["HomeCredit_columns_description.csv"]])
ls_files

[u'POS_CASH_balance.csv',
 u'application_test.csv',
 u'application_train.csv',
 u'bureau.csv',
 u'bureau_balance.csv',
 u'credit_card_balance.csv',
 u'installments_payments.csv',
 u'previous_application.csv',
 u'sample_submission.csv']

In [4]:
def exploring_stats(pdf_input):
    # check rows, cols
    total_records = pdf_input.shape[0]
    total_columns = pdf_input.shape[1]
    print("Total records:", total_records)
    print("Total columns:", total_columns)

    # check dtypes
    name = []
    sub_type = []
    for n, t in pdf_input.dtypes.iteritems():
        name.append(n)
        sub_type.append(t)

    # check distinct
    ls_ndist = []
    for cname in pdf_input.columns:
        ndist = pdf_input[cname].nunique()
        pct_dist = ndist * 100.0 / total_records
        ls_ndist.append("{} ({:0.2f}%)".format(ndist, pct_dist))

    # check missing
    ls_nmiss = []
    for cname in pdf_input.columns:
        nmiss = pdf_input[cname].isnull().sum()
        pct_miss = nmiss * 100.0 / total_records
        ls_nmiss.append("{} ({:0.2f}%)".format(nmiss, pct_miss))

    # check zeros
    ls_zeros = []
    for cname in pdf_input.columns:
        try:
            nzeros = (pdf_input[cname] == 0).sum()
            pct_zeros = nzeros * 100.0 / total_records
            ls_zeros.append("{} ({:0.2f}%)".format(nzeros, pct_zeros))
        except:
            ls_zeros.append("{} ({:0.2f}%)".format(0, 0))
            continue

    # check negative
    ls_neg = []
    for cname in pdf_input.columns:
        try:
            nneg = (pdf_input[cname].astype("float") < 0).sum()
            pct_neg = nneg * 100.0 / total_records
            ls_neg.append("{} ({:0.2f}%)".format(nneg, pct_neg))
        except:
            ls_neg.append("{} ({:0.2f}%)".format(0, 0))
            continue

    # prepare output
    data = {
        "name": name,
        "sub_type": sub_type,
        "n_distinct": ls_ndist,
        "n_miss": ls_nmiss,
        "n_zeros": ls_zeros,
        "n_negative": ls_neg,
    }

    # check stats
    pdf_stats = pdf_input.describe().transpose()
    ls_stats = []
    for stat in pdf_stats.columns:
        data[stat] = []
        for cname in pdf_input.columns:
            try:
                data[stat].append(pdf_stats.loc[cname, stat])
            except:
                data[stat].append(0.0)

    # take samples
    nsample = 10
    pdf_sample = pdf_input.sample(frac=.5).head(nsample).transpose()
    pdf_sample.columns = ["sample_{}".format(i) for i in range(nsample)]

    # output
    col_ordered = ["sub_type", "n_distinct", "n_miss", "n_negative", "n_zeros",
                   "25%", "50%", "75%", "count", "max", "mean", "min", "std"] + list(pdf_sample.columns)
    pdf_data = pd.DataFrame(data).set_index("name")
    pdf_data = pd.concat([pdf_data, pdf_sample], axis=1)
    pdf_data = pdf_data[col_ordered]

    return pdf_data

In [5]:
%%time
ls_report = {}
for f in ls_files:
    print("Exploring {}".format(f))
    data_path = os.path.join("home-credit-default-risk", f)
    pdf_data = pd.read_csv(data_path)
    ls_report[f] = exploring_stats(pdf_data)
    display(ls_report[f])

Exploring POS_CASH_balance.csv
('Total records:', 10001358)
('Total columns:', 8)


,sub_type,n_distinct,n_miss,n_negative,n_zeros,25%,50%,75%,count,max,mean,min,std,sample_0,sample_1,sample_2,sample_3,sample_4,sample_5,sample_6,sample_7,sample_8,sample_9
name,,,,,,,,,,,,,,,,,,,,,,,
SK_ID_PREV,int64,936325 (9.36%),0 (0.00%),0 (0.00%),0 (0.00%),1434405.0,1896565.0,2368963.0,1.0001e+07,2843499.0,1.9032e+06,1000001.0,535846.5307,2749119,1439762,2404083,1417828,1975679,2465369,2059772,1570568,2313395,2556181
SK_ID_CURR,int64,337252 (3.37%),0 (0.00%),0 (0.00%),0 (0.00%),189550.0,278654.0,367429.0,1.0001e+07,456255.0,2.7840e+05,100001.0,102763.7451,136725,412165,394468,230896,290188,301648,415723,432016,154158,280427
MONTHS_BALANCE,int64,96 (0.00%),0 (0.00%),10001358 (100.00%),0 (0.00%),-54.0,-28.0,-13.0,1.0001e+07,-1.0,-3.5013e+01,-96.0,26.0666,-40,-11,-24,-6,-63,-12,-2,-92,-21,-22
CNT_INSTALMENT,float64,73 (0.00%),26071 (0.26%),0 (0.00%),0 (0.00%),10.0,12.0,24.0,9.9753e+06,92.0,1.7090e+01,1.0,11.9951,12,12,10,8,10,36,12,10,60,18
CNT_INSTALMENT_FUTURE,float64,79 (0.00%),26087 (0.26%),0 (0.00%),1185960 (11.86%),3.0,7.0,14.0,9.9753e+06,85.0,1.0484e+01,0.0,11.1091,1,3,10,8,1,17,1,10,51,17
NAME_CONTRACT_STATUS,object,9 (0.00%),0 (0.00%),0 (0.00%),0 (0.00%),0.0,0.0,0.0,0.0000e+00,0.0,0.0000e+00,0.0,0.0000,Active,Active,Active,Active,Active,Active,Active,Active,Active,Active
SK_DPD,int64,3400 (0.03%),0 (0.00%),0 (0.00%),9706131 (97.05%),0.0,0.0,0.0,1.0001e+07,4231.0,1.1607e+01,0.0,132.7140,0,0,0,0,0,0,0,0,0,0
SK_DPD_DEF,int64,2307 (0.02%),0 (0.00%),0 (0.00%),9887389 (98.86%),0.0,0.0,0.0,1.0001e+07,3595.0,6.5447e-01,0.0,32.7625,0,0,0,0,0,0,0,0,0,0


Exploring application_test.csv
('Total records:', 48744)
('Total columns:', 121)


,sub_type,n_distinct,n_miss,n_negative,n_zeros,25%,50%,75%,count,max,mean,min,std,sample_0,sample_1,sample_2,sample_3,sample_4,sample_5,sample_6,sample_7,sample_8,sample_9
name,,,,,,,,,,,,,,,,,,,,,,,
SK_ID_CURR,int64,48744 (100.00%),0 (0.00%),0 (0.00%),0 (0.00%),188557.7500,277549.0000,367555.5000,48744.0,4.5625e+05,2.7780e+05,1.0000e+05,103169.5473,413769,267284,260484,359319,152020,211553,183878,303497,285833,261749
NAME_CONTRACT_TYPE,object,2 (0.00%),0 (0.00%),0 (0.00%),0 (0.00%),0.0000,0.0000,0.0000,0.0,0.0000e+00,0.0000e+00,0.0000e+00,0.0000,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans
CODE_GENDER,object,2 (0.00%),0 (0.00%),0 (0.00%),0 (0.00%),0.0000,0.0000,0.0000,0.0,0.0000e+00,0.0000e+00,0.0000e+00,0.0000,F,F,F,F,F,M,F,M,F,F
FLAG_OWN_CAR,object,2 (0.00%),0 (0.00%),0 (0.00%),0 (0.00%),0.0000,0.0000,0.0000,0.0,0.0000e+00,0.0000e+00,0.0000e+00,0.0000,Y,N,N,N,N,N,Y,Y,N,N
FLAG_OWN_REALTY,object,2 (0.00%),0 (0.00%),0 (0.00%),0 (0.00%),0.0000,0.0000,0.0000,0.0,0.0000e+00,0.0000e+00,0.0000e+00,0.0000,Y,N,Y,N,N,N,N,Y,N,Y
CNT_CHILDREN,int64,11 (0.02%),0 (0.00%),0 (0.00%),34685 (71.16%),0.0000,0.0000,1.0000,48744.0,2.0000e+01,3.9705e-01,0.0000e+00,0.7090,0,0,0,0,0,0,0,1,0,0
AMT_INCOME_TOTAL,float64,606 (1.24%),0 (0.00%),0 (0.00%),0 (0.00%),112500.0000,157500.0000,225000.0000,48744.0,4.4100e+06,1.7843e+05,2.6942e+04,101522.5915,1.125e+05,9e+04,3.375e+05,1.8e+05,9e+04,1.35e+05,1.125e+05,2.475e+05,6.75e+04,8.1e+04
AMT_CREDIT,float64,2937 (6.03%),0 (0.00%),0 (0.00%),0 (0.00%),260640.0000,450000.0000,675000.0000,48744.0,2.2455e+06,5.1674e+05,4.5000e+04,365397.0042,3.769e+05,2.725e+05,1.305e+06,3.6e+05,2.37e+05,2.7e+05,6.75e+05,3.820e+05,6.75e+05,3.644e+05
AMT_ANNUITY,float64,7491 (15.37%),24 (0.05%),0 (0.00%),0 (0.00%),17973.0000,26199.0000,37390.5000,48720.0,1.8058e+05,2.9426e+04,2.2950e+03,16016.3683,1.937e+04,2.166e+04,4.21e+04,2.202e+04,1.528e+04,2.476e+04,3.46e+04,2.35e+04,2.545e+04,1.545e+04


Exploring application_train.csv
('Total records:', 307511)
('Total columns:', 122)


...

Exploring previous_application.csv
('Total records:', 1670214)
('Total columns:', 37)


,sub_type,n_distinct,n_miss,n_negative,n_zeros,25%,50%,75%,count,max,mean,min,std,sample_0,sample_1,sample_2,sample_3,sample_4,sample_5,sample_6,sample_7,sample_8,sample_9
name,,,,,,,,,,,,,,,,,,,,,,,
SK_ID_PREV,int64,1670214 (100.00%),0 (0.00%),0 (0.00%),0 (0.00%),1.4619e+06,1.9231e+06,2.3843e+06,1670214.0,2.8454e+06,1.9231e+06,1.0000e+06,532597.9587,1092395,1252745,1426776,1649070,1937880,2786742,2369053,2100244,2751684,1685570
SK_ID_CURR,int64,338857 (20.29%),0 (0.00%),0 (0.00%),0 (0.00%),1.8933e+05,2.7871e+05,3.6751e+05,1670214.0,4.5626e+05,2.7836e+05,1.0000e+05,102814.8238,334586,307326,164677,289639,316386,325443,329812,151568,184899,428430
NAME_CONTRACT_TYPE,object,4 (0.00%),0 (0.00%),0 (0.00%),0 (0.00%),0.0000e+00,0.0000e+00,0.0000e+00,0.0,0.0000e+00,0.0000e+00,0.0000e+00,0.0000,Revolving loans,Cash loans,Cash loans,Consumer loans,Consumer loans,Cash loans,Consumer loans,Revolving loans,Cash loans,Consumer loans
AMT_ANNUITY,float64,357959 (21.43%),372235 (22.29%),0 (0.00%),1637 (0.10%),6.3218e+03,1.1250e+04,2.0658e+04,1297979.0,4.1806e+05,1.5955e+04,0.0000e+00,14782.1373,2250,NaN,NaN,6263,7556,NaN,1.26e+04,2.025e+04,2.758e+04,1.289e+04
AMT_APPLICATION,float64,93885 (5.62%),0 (0.00%),0 (0.00%),392402 (23.49%),1.8720e+04,7.1046e+04,1.8036e+05,1670214.0,6.9052e+06,1.7523e+05,0.0000e+00,292779.7624,4.5e+04,0,0,4.364e+04,7.308e+04,0,1.35e+05,0,4.5e+05,1.179e+05
AMT_CREDIT,float64,86803 (5.20%),1 (0.00%),0 (0.00%),336768 (20.16%),2.4160e+04,8.0541e+04,2.1642e+05,1670213.0,6.9052e+06,1.9611e+05,0.0000e+00,318574.6165,4.5e+04,0,0,5.328e+04,7.308e+04,0,1.35e+05,4.05e+05,4.916e+05,1.146e+05
AMT_DOWN_PAYMENT,float64,29278 (1.75%),895844 (53.64%),2 (0.00%),369854 (22.14%),0.0000e+00,1.6380e+03,7.7400e+03,774370.0,3.0600e+06,6.6974e+03,-9.0000e-01,20921.4954,NaN,NaN,NaN,0,0,NaN,0,NaN,NaN,2.357e+04
AMT_GOODS_PRICE,float64,93885 (5.62%),385515 (23.08%),0 (0.00%),6869 (0.41%),5.0841e+04,1.1232e+05,2.3400e+05,1284699.0,6.9052e+06,2.2785e+05,0.0000e+00,315396.5579,4.5e+04,NaN,NaN,4.364e+04,7.308e+04,NaN,1.35e+05,NaN,4.5e+05,1.179e+05
WEEKDAY_APPR_PROCESS_START,object,7 (0.00%),0 (0.00%),0 (0.00%),0 (0.00%),0.0000e+00,0.0000e+00,0.0000e+00,0.0,0.0000e+00,0.0000e+00,0.0000e+00,0.0000,WEDNESDAY,FRIDAY,WEDNESDAY,SUNDAY,SATURDAY,FRIDAY,TUESDAY,MONDAY,SATURDAY,FRIDAY


Exploring sample_submission.csv
('Total records:', 48744)
('Total columns:', 2)


,sub_type,n_distinct,n_miss,n_negative,n_zeros,25%,50%,75%,count,max,mean,min,std,sample_0,sample_1,sample_2,sample_3,sample_4,sample_5,sample_6,sample_7,sample_8,sample_9
name,,,,,,,,,,,,,,,,,,,,,,,
SK_ID_CURR,int64,48744 (100.00%),0 (0.00%),0 (0.00%),0 (0.00%),188557.75,277549.0,367555.5,48744.0,456250.0,277796.6763,100001.0,103169.5473,262887.0,279679.0,340511.0,256663.0,261967.0,359353.0,364403.0,423087.0,115680.0,370913.0
TARGET,float64,1 (0.00%),0 (0.00%),0 (0.00%),0 (0.00%),0.50,0.5,0.5,48744.0,0.5,0.5000,0.5,0.0000,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


CPU times: user 6min 36s, sys: 16 s, total: 6min 52s
Wall time: 1min 39s


In [7]:
%%time
# save reports
for k, v in ls_report.items():
    print("Saving {}".format(k))
    ls_report[k].to_csv(os.path.join("reports", "report_{}".format(k)))

Saving previous_application.csv
Saving application_test.csv
Saving installments_payments.csv
Saving sample_submission.csv
Saving credit_card_balance.csv
Saving bureau.csv
Saving application_train.csv
Saving POS_CASH_balance.csv
Saving bureau_balance.csv
CPU times: user 14.3 ms, sys: 8.26 ms, total: 22.6 ms
Wall time: 26.1 ms
